In [ ]:
import sys
sys.path.append('../python_packages_static/')
import pyemu
import os
import matplotlib.pyplot as plt

## run NOPTMAX=0 to get residuals, then load and reweight like we were doing for Monte Carlo. But, also set up standard deviation column for noise on observations for the ensembles

In [ ]:
input_dir = '../noptmax0_testing/' # read the pst file from here and get residuals
output_dir = '../run_data'  # write out the new run file here
pst_root = 'never_iES_noptmax0' # the PST root we are reading in from

ies_noise_obs_file = 'never_ies_0.noise_lt_obs.pst' # Name of the PST file to write out as the start of iES

### read in the PEST control file

In [ ]:
pst = pyemu.Pst(os.path.join(input_dir,'{}.pst'.format(pst_root)))

### make a quick pie chart showing the current distribution of the observation groups in the objective function

In [ ]:
pst.plot(kind='phi_pie')
plt.savefig('../report_materials/post_mc_noptmax0_phi_pie.pdf', dpi=600)

### we can use `pyemu` functionality to assign new weights that adjust and honor whatever balance we seek

In [ ]:
new_proportions = pst.phi_components.copy()
new_proportions

### here we assign proportions (that sum to 1.0) to the various groups. We want to retain the same total Phi so we multiply our proportions by the total Phi

In [ ]:
new_proportions['flux'] = 0.3*pst.phi
new_proportions['head'] = 0.2*pst.phi
new_proportions['land_surface'] = 0.5*pst.phi
new_proportions['budget'] = 0

In [ ]:
new_proportions

### using the `phi_components` dictionary, making a copy of it, and reassigning values, we can update the PST object using the `adjust_weights` function

In [ ]:
pst.adjust_weights(obsgrp_dict=new_proportions)

In [ ]:
pst.observation_data

In [ ]:
pst.plot(kind='phi_pie')
plt.savefig('../report_materials/never_ies_0.noise_lt_obs_phi_pie.pdf', dpi=600)

In [ ]:
# set some values for ies
pst.pestpp_options["ies_num_reals"] = 500  
pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_save_rescov"] = True
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 3.0
pst.pestpp_options['ies_autoadaloc']=True
pst.pestpp_options['ies_num_threads']=4
pst.pestpp_options['ies_lambda_mults']=(0.1,1.0,10.0,100.0)
pst.pestpp_options['lambda_scale_fac'] = (0.75,0.9,1.0,1.1)
pst.pestpp_options['ies_subset_size']=10

# set SVD for some regularization
pst.svd_data.maxsing = 250

### we want to use the observations in the `land_surface` group as inequality observations (e.g. unweighted unless needed to enforce "less than" condition). By naming the groups starting with "less_than" PESTPP-IES will interpret them as such

In [ ]:
pst.observation_data.loc[pst.observation_data.obgnme=='land_surface', 'obgnme'] = 'less_than_land_surface' # rename for less than inequality

In [ ]:
pst.observation_data

# Add standard deviation column for observations to control noise realizations

### make a "standard_deviation" column in the pst observation_data dataframe. This will inform PESTPP-IES to use these value for sampling in generating the observation noise on the ensembles. 

Start out with a value of -9999 that we can use as a test to make sure we set a value for every observation group.

In [ ]:
obs = pst.observation_data
obs['standard_deviation'] = -9999

### now we set some subjective standard deviation values that we assert are more appropriate than the straight up weights.

In [ ]:
obs.loc[obs.obsnme=='q_1436500'].obsval/(3.33*3)

In [ ]:
obs.loc[obs.obsnme=='q_1366650'].obsval/30

In [ ]:
obs.loc[obs.obsnme=='q_1436500', 'standard_deviation'] = obs.loc[obs.obsnme=='q_1436500'].obsval/(3.33*3)
obs.loc[obs.obsnme=='q_1366650', 'standard_deviation'] = obs.loc[obs.obsnme=='q_1366650'].obsval/(10.*3)

In [ ]:
obs.loc[obs.obgnme=='head', 'standard_deviation'] =  5/3  #  5 is the range, div by 3 for the range
obs.loc[obs.obgnme=='less_than_land_surface', 'standard_deviation'] = 10/3
obs.loc[obs.obgnme=='budget', 'standard_deviation'] = 9999

### use an `assert` statment to make sure we set obs to have a standard deviation not -9999

In [ ]:
assert pst.observation_data.standard_deviation.min()>0

In [ ]:
pst.observation_data

### Write out an updated parameter summary XLS file

In [ ]:
parsum = pst.write_par_summary_table('../report_materials/initial_iES_parsum.xlsx', report_in_linear_space=True)
parsum

### Now set `NOPTMAX=10` and write out the PEST control file. Note - to use the standard deviation column requires writing out in the new `version=2` format. In this format, the parameter and observation data sections are written to external files. This is not backward compatible with older version of PEST, but enables this new capability. Filenames for the external files will be identified in the PST file.

In [ ]:
pst.control_data.noptmax = 10
pst.pestpp_options["ies_no_noise"] = False
pst.write(os.path.join(output_dir, ies_noise_obs_file), version=2)